In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input
import pandas as pd
from gamspy import Options

In [2]:
data = read_input("toy_problem.xlsx")

In [3]:
m = GatheringModel("Hola mundo")

In [4]:
m.instance_model(data)

In [5]:
container = m.m

In [10]:
container["capacity"].records

,s,c,value
0,s1,oil,10000.0
1,s1,gas,30000.0
2,s1,water,30000.0
3,s2,oil,20000.0
4,s2,gas,60000.0
5,s2,water,60000.0


In [12]:
container["maxFlow"].records

In [9]:
container["Qprod"].records

,t,value
0,t1,6500.0
1,t2,6100.0
2,t3,5200.0
3,t4,4400.0
4,t5,4000.0
...,...,...
115,t116,205.0
116,t117,203.0
117,t118,202.0
118,t119,200.0


In [6]:
model = container.models["Multiphase_network_design"]

In [7]:
# summary = model.solve(solver="conopt", options=Options(iteration_limit=2), solver_options={"rtmaxv": "1.e12"})
summary = model.solve(solver="cplex", options=Options(equation_listing_limit=100))

In [8]:
summary

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,IntegerInfeasible,NaN,1054729,1132441,MIP,CPLEX,0.75


In [38]:
model.getEquationListing()[:200]

'mass_balance(i1,oil,t15)..  - Qinter(i1,j1,d1,t15,oil) - Qinter(i1,j1,d2,t15,oil) - Qinter(i1,j1,d3,t15,oil) - Qinter(i1,j1,d4,t15,oil) - Qinter(i1,j2,d1,t15,oil) - Qinter(i1,j2,d2,t15,oil) - Qinter(i'